In [ ]:
import pandas as pd
import torch
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

In [ ]:
# --- 1. CARREGAR O MODELO E OS DADOS NÃO ROTULADOS ---

# Carregar o modelo que acabamos de treinar
model_path = "./models/leia_classifier_v1_final"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Carregar a amostra de trabalho completa
df_working_set = pd.read_csv('/Users/giossaurus/Developer/leia_tcc/data/processed/amostra_trabalho_1000.csv')

# Carregar o seed set que já foi anotado para podermos excluí-lo
df_seed_set = pd.read_csv('/Users/giossaurus/Developer/leia_tcc/data/processed/seed_set_anotado.csv')

# Filtrar para obter apenas os dados que AINDA NÃO foram anotados
unlabeled_df = df_working_set[~df_working_set['question_id'].isin(df_seed_set['question_id'])]
unlabeled_df = unlabeled_df.rename(columns={'question': 'text'})

print(f"Encontrados {len(unlabeled_df)} exemplos não rotulados para análise.")

# Converter para o formato do Hugging Face
unlabeled_dataset = Dataset.from_pandas(unlabeled_df)

In [ ]:
# --- 2. FAZER PREDIÇÕES E CALCULAR A INCERTEZA ---

# Tokenizar os dados não rotulados
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_unlabeled_dataset = unlabeled_dataset.map(tokenize_function, batched=True)

# Criar um 'Trainer' apenas para fazer predições (não vamos treinar aqui)
trainer = Trainer(model=model)

# Obter as predições (logits) do modelo
predictions = trainer.predict(tokenized_unlabeled_dataset)
logits = predictions.predictions

# Converter logits para probabilidades usando a função softmax
probabilities = torch.nn.functional.softmax(torch.from_numpy(logits), dim=-1).numpy()

# Calcular a incerteza. Uma métrica simples e eficaz é a "menor margem",
# que é a diferença entre as duas classes mais prováveis.
# Quanto MENOR a margem, MAIS INCERTO o modelo está.
sorted_probabilities = np.sort(probabilities, axis=1)
margin = sorted_probabilities[:, -1] - sorted_probabilities[:, -2] # Pega a maior prob - a segunda maior

# Adicionar a incerteza ao nosso dataframe
unlabeled_df['uncertainty'] = 1 - margin # Invertemos para que maiores valores sejam mais incertos

In [ ]:
# --- 3. EXIBIR AS PERGUNTAS PARA A PRÓXIMA ANOTAÇÃO ---

# Ordenar o dataframe pela incerteza e pegar as 50 mais incertas
N_SAMPLES_TO_ANNOTATE = 50
df_to_annotate = unlabeled_df.sort_values(by='uncertainty', ascending=False).head(N_SAMPLES_TO_ANNOTATE)

print(f"\n--- AS {N_SAMPLES_TO_ANNOTATE} PERGUNTAS MAIS INCERTAS PARA VOCÊ ANOTAR: ---")

# Exibir as perguntas de forma legível
for index, row in df_to_annotate.iterrows():
    print(f"--- Pergunta ID: {row['question_id']} (Incerteza: {row['uncertainty']:.2f}) ---")
    print(row['text'])
    print("-" * 50)

# Salvar este lote para anotação no Label Studio
df_to_annotate.to_csv('/Users/giossaurus/Developer/leia_tcc/data/processed/active_learning_cycle1_to_annotate.csv', index=False)
print(f"\nArquivo com as {N_SAMPLES_TO_ANNOTATE} perguntas salvo em 'data/processed/active_learning_cycle1_to_annotate.csv'")

Encontrados 801 exemplos não rotulados para análise.


Map:   0%|          | 0/801 [00:00<?, ? examples/s]

/opt/homebrew/Caskroom/miniforge/base/envs/torch-mps/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)



--- AS 50 PERGUNTAS MAIS INCERTAS PARA VOCÊ ANOTAR: ---
--- Pergunta ID: 49d74598320b63455092e4a3ca2d669363b50fc23e2fb951e3da0b596fce34303c66294ea3f67bc79461d82df3d1fecfc709243c462da654a302dbf9ef9edfd7 (Incerteza: 1.00) ---
TEXTO DE REFERÊNCIA: A Islândia é um dos únicos lugares do mundo onde um limite de placa divergente é exposto na superfície da Terra e é uma das regiões vulcânicas mais ativas do planeta. O hotspot logo abaixo do país é a causa da alta atividade vulcânica, e alguns cientistas acreditam que também pode ter contribuído para o rompimento do supercontinente Pangeia e a subsequente formação do Oceano Atlântico Norte. Rochas ígneas que surgiram deste hotspot foram encontradas em ambos os lados da Dorsal Meso-Atlântica, que se originou há aproximadamente 57 milhões de anos.

Qual processo geológico é gerado por essa atividade vulcânica?
--------------------------------------------------
--- Pergunta ID: be140d4b96594613deffc929bf83425e4fea611f35ce850fb677aa397f3d8f1c3dd55